# Movie Analysis

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

from os import listdir
from os.path import isfile, join
import pandas as pd
import json

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [57]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

## Data Collection
Data about movies is in no short supply on the web. After deciding on which data would be most relevant, we found two data sets that would complement each other 

Kaggle - https://www.kaggle.com/tmdb/tmdb-movie-metadata

Load the data from the raw files into Pandas dataframes. Some preprocessing had to be done on these files since they had some non-utf-8 characters.

In [43]:
file_dir = "Data/RawData/"

data_files = [f for f in listdir(file_dir) if isfile(join(file_dir, f))]
data_list = []

for data_file in data_files:
    if "Store" not in data_file:
        data_list.append(pd.read_csv(file_dir + data_file, engine='python'))

merged = data_list[1].join(data_list[2], lsuffix='title', rsuffix='title')
data_list[0].drop(['movie_id'], axis=1)
merged = merged.join(data_list[0], lsuffix='title', rsuffix='title')

merged.rename(index=str, columns={"titletitle": "title"}, inplace=True)
merged = merged.loc[:,~merged.columns.duplicated()] # Remove duplicate columns from joining

merged.head()


,keywords,production_companies,production_countries,release_date,revenue,title,vote_average,vote_count,color,director_name,...,plot_keywords,num_user_for_reviews,language,content_rating,actor_2_facebook_likes,aspect_ratio,movie_facebook_likes,movie_id,cast,crew
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",12/10/09,2787965087,Avatar,7.2,11800,Color,James Cameron,...,avatar|future|marine|native|paraplegic,3054.0,English,PG-13,936.0,1.78,33000,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",5/19/07,961000000,Pirates of the Caribbean: At World's End,6.9,4500,Color,Gore Verbinski,...,goddess|marriage ceremony|marriage proposal|pi...,1238.0,English,PG-13,5000.0,2.35,0,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",10/26/15,880674609,Spectre,6.3,4466,Color,Sam Mendes,...,bomb|espionage|sequel|spy|terrorist,994.0,English,PG-13,393.0,2.35,85000,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/16/12,1084939099,The Dark Knight Rises,7.6,9106,Color,Christopher Nolan,...,deception|imprisonment|lawlessness|police offi...,2701.0,English,PG-13,23000.0,2.35,164000,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3/7/12,284139100,John Carter,6.1,2124,Color,Andrew Stanton,...,alien|american civil war|male nipple|mars|prin...,738.0,English,PG-13,632.0,2.35,24000,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Countries that a movie was produced is some interesting data. Flatten it down and one-hot encode it.

In [44]:
rows = []
for i, row in enumerate(merged['production_countries']):
    if len(json.loads(row)) > 0:
        rows.append("|".join([country['name'] for country in json.loads(row)]))
    else:
        rows.append("")
merged.drop(['production_countries'], axis=1)
se = pd.Series(rows)
merged['production_countries'] = se.values
cleaned_pc = merged.production_countries.str.split('|', expand=True).stack()
production_countries = pd.get_dummies(cleaned_pc, prefix='countryProduced').groupby(level=0).sum()

One hot encode genres, color, content rating, and language

In [45]:
cleaned_g = merged.genres.str.split('|', expand=True).stack()
genres = pd.get_dummies(cleaned_g, prefix='g').groupby(level=0).sum()

color = pd.get_dummies(merged['color'], prefix='color')
content_rating = pd.get_dummies(merged['content_rating'], prefix='rating')
language = pd.get_dummies(merged['language'], prefix='language')


Merge in all of the one-hot encoded data into the merged dataframe

In [46]:
# Add the one-hot encoded dataframes to the final set
merged = pd.concat([merged, genres], axis=1, sort=True)
merged = pd.concat([merged, color], axis=1, sort=True)
merged = pd.concat([merged, content_rating], axis=1, sort=True)
merged = pd.concat([merged, language], axis=1, sort=True)
merged = pd.concat([merged, production_countries], axis=1, sort=True)

Dropping data that will either expand the data too much if we one-hot encode, is duplicates of other data we have, or we have already one-hot encoded.

In [47]:
merged = merged.drop(['genres', 'movie_title', 'gross', 'production_countries', 'num_user_for_reviews',
                      'director_name', 'actor_2_name', 'actor_1_facebook_likes',
                      'actor_1_name', 'actor_3_name', 'color', 'content_rating', 'language'], axis=1)

Drop other columns

In [48]:
# Drop columns that we will not be using yet, but might later (keywords)
merged = merged.drop(['plot_keywords', 'keywords'], axis=1)

# I think production companies might be a bit much. lets remove it for now.
merged = merged.drop(['production_companies'], axis=1)

### Fix the release date (https://stackoverflow.com/questions/46428870/how-to-handle-date-variable-in-machine-learning-data-pre-processing)
### Dropping for now, but this is definitely something to do feature engineering on!

### Thought: I wonder if we can get data on opening week or month revenue


In [49]:
merged = merged.drop(['release_date'], axis=1)
merged.head()

,revenue,title,vote_average,vote_count,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,num_voted_users,cast_total_facebook_likes,...,countryProduced_Sweden,countryProduced_Switzerland,countryProduced_Taiwan,countryProduced_Thailand,countryProduced_Tunisia,countryProduced_Turkey,countryProduced_Ukraine,countryProduced_United Arab Emirates,countryProduced_United Kingdom,countryProduced_United States of America
0,2787965087,Avatar,7.2,11800,723.0,178.0,0.0,855.0,886204,4834,...,0,0,0,0,0,0,0,0,1,1
1,961000000,Pirates of the Caribbean: At World's End,6.9,4500,302.0,169.0,563.0,1000.0,471220,48350,...,0,0,0,0,0,0,0,0,0,1
10,391081192,Superman Returns,5.4,1400,434.0,169.0,0.0,903.0,240396,29991,...,0,0,0,0,0,0,0,0,0,1
100,333932083,The Curious Case of Benjamin Button,7.3,3292,362.0,166.0,21000.0,919.0,459346,13333,...,0,0,0,0,0,0,0,0,0,1
1000,28931401,Drive Angry,5.3,600,298.0,125.0,487.0,201.0,92461,1300,...,0,0,0,0,0,0,0,0,0,1


## Thought: We should be able to use the credit data for the movies we have. The following features should help: Crew Size, number of position (for each position), 

In [50]:
merged = merged.drop(['cast', 'crew'], axis=1)

Clean up any nan or nonzero values

In [51]:
# set to average any 0 or nan for most of the continous columns
# 0 or nan: revenue, duration
zero_or_nan_average = ['revenue', 'duration']
for col in zero_or_nan_average:
    mean = merged[col].mean()
    merged[col].fillna((mean), inplace=True)
    merged = merged.replace({col: {0: mean}})
    
just_nan_average = ['vote_average', 'vote_count', 'num_critic_for_reviews', 'director_facebook_likes', 'actor_3_facebook_likes', 'num_voted_users',
                    'cast_total_facebook_likes', 'facenumber_in_poster', 'actor_2_facebook_likes', 'movie_facebook_likes']
for col in just_nan_average:
    merged[col].fillna((merged[col].mean()), inplace=True)
    
# Remove anything that wasn't fixed
merged = merged.dropna()
merged = merged.set_index('title')
merged.head()

,revenue,vote_average,vote_count,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,...,countryProduced_Sweden,countryProduced_Switzerland,countryProduced_Taiwan,countryProduced_Thailand,countryProduced_Tunisia,countryProduced_Turkey,countryProduced_Ukraine,countryProduced_United Arab Emirates,countryProduced_United Kingdom,countryProduced_United States of America
title,,,,,,,,,,,,,,,,,,,,,
Avatar,2.787965e+09,7.2,11800,723.0,178.0,0.0,855.0,886204,4834,0.0,...,0,0,0,0,0,0,0,0,1,1
Pirates of the Caribbean: At World's End,9.610000e+08,6.9,4500,302.0,169.0,563.0,1000.0,471220,48350,0.0,...,0,0,0,0,0,0,0,0,0,1
Superman Returns,3.910812e+08,5.4,1400,434.0,169.0,0.0,903.0,240396,29991,0.0,...,0,0,0,0,0,0,0,0,0,1
The Curious Case of Benjamin Button,3.339321e+08,7.3,3292,362.0,166.0,21000.0,919.0,459346,13333,2.0,...,0,0,0,0,0,0,0,0,0,1
Drive Angry,2.893140e+07,5.3,600,298.0,125.0,487.0,201.0,92461,1300,3.0,...,0,0,0,0,0,0,0,0,0,1


We have some rows that make this task too easy. We want this model to be able to predict revenue BEFORE release, and not after. Let's remove features that we would only obtain after release.

In [52]:
merged = merged.drop(["vote_average", "vote_count", "num_critic_for_reviews", "num_voted_users", "movie_facebook_likes"], axis=1)


I hate that this needs to be removed, but the facebook likes metric introduces bias into our dataset, since older movies are very likely to have very little if any facebook likes. It would have been such good data though!

In [53]:
merged = merged.drop(["director_facebook_likes", "actor_3_facebook_likes", "cast_total_facebook_likes", "actor_2_facebook_likes"], axis=1)


Wrap up the data preprocessing!

In [54]:
msk = np.random.rand(len(merged)) < 0.8
train = merged[msk]
test = merged[~msk]

train.to_csv(r'Data/train.csv', index = train.index.tolist(), header=True)
test.drop("revenue", axis=1).to_csv(r'Data/test.csv', index = test.index.tolist(), header=True)
test.to_csv(r'Data/test_sol.csv', index = test.index.tolist(), columns = ["revenue"])

## On to the ML!

In [55]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

In [56]:
#log transform the target:
train["revenue"] = np.log1p(train["revenue"])

# log transform skewed numeric features:
numeric_feats = train.dtypes[train.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

train[skewed_feats] = np.log1p(train[skewed_feats])
test[skewed_feats] = np.log1p(test[skewed_feats])

#creating matrices for sklearn:
X_train = train
X_test = test
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
y = train.revenue
X_train = X_train.set_index("title")
X_test = X_test.set_index("title")
X_train = X_train.drop("revenue", axis=1)

print("Test Length: " + str(len(X_test)))
print("Train Length: " + str(len(X_train)))
X_test.head()


Test Length: 915
Train Length: 3627


,duration,facenumber_in_poster,aspect_ratio,movie_id,g_Action,g_Adventure,g_Animation,g_Biography,g_Comedy,g_Crime,...,countryProduced_Sweden,countryProduced_Switzerland,countryProduced_Taiwan,countryProduced_Thailand,countryProduced_Tunisia,countryProduced_Turkey,countryProduced_Ukraine,countryProduced_United Arab Emirates,countryProduced_United Kingdom,countryProduced_United States of America
title,,,,,,,,,,,,,,,,,,,,,
Drive Angry,4.836282,1.386294,1.208960,10.764857,0.693147,0.693147,0.0,0.000000,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
The Tooth Fairy,4.744932,1.098612,1.208960,10.895887,0.693147,0.000000,0.0,0.000000,0,0.693147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3:10 to Yuma,4.912655,0.693147,1.208960,8.551981,0.000000,0.000000,0.0,0.000000,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
Taken 3,4.691348,1.098612,1.208960,12.469771,0.000000,0.000000,0.0,0.693147,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
Dick Tracy,4.564348,0.693147,1.047319,9.058703,0.693147,0.000000,0.0,0.000000,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### Lasso Regression

In [58]:
model_lasso = LassoCV(alphas = alphas, tol=0.1, cv=3).fit(X_train, y)
rmse_cv(model_lasso).mean()

1.780391166265089

In [59]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

print("")
print("Top Positive Influencers:")
print(coef.nlargest(35))
print("")
print("Top Negative Influencers:")
print(coef.nsmallest(35))


Lasso picked 17 variables and eliminated the other 167 variables

Top Positive Influencers:
duration                                    0.717940
g_Adventure                                 0.592385
g_Action                                    0.461843
rating_PG-13                                0.410550
g_Animation                                 0.397034
g_Family                                    0.310784
countryProduced_United States of America    0.198030
g_Fantasy                                   0.175947
language_English                            0.157668
g_Sci-Fi                                    0.112516
aspect_ratio                                0.093666
facenumber_in_poster                        0.041578
rating_PG                                   0.025050
g_Biography                                 0.000000
g_Crime                                     0.000000
g_Documentary                              -0.000000
g_Film-Noir                                -0.000000
g_Game-

In [60]:
lasso_preds = np.expm1(model_lasso.predict(X_test))
solution = pd.DataFrame({"title":X_test.index.tolist(), "revenue":lasso_preds})
solution.to_csv("Solutions/lasso_sol.csv", index = False)